<a href="https://colab.research.google.com/github/ShawnFromChineseTaipei/class_test/blob/main/XGBRFSVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks')
os.listdir()

Mounted at /content/drive


['XGBRFSVR.ipynb',
 'Untitled0.ipynb',
 '「day-one.ipynb」的副本',
 'Transportation DA.ipynb',
 'DA note.ipynb',
 '「day-two.ipynb」的副本 (1)',
 '「day-two.ipynb」的副本']

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
data = pd.read_csv("/content/Resin_5HB3.csv")
data

,WorkNumber,BarrelBatch,MixingTime1,MixingTime2,MixingTime3,TempSet,SandDischatge_Tem,Packaging_Temp,SandWeight_E,SandWeight_L,...,Hardener_40,Water,Weight_Calcium,Weight_RedIron,Aumosphere_Temp,Atmosphere_humidty,GrainSize\n(AFS),AFS,UnityPoint,FallingTest
0,1.100000e+11,1,50,60,20,130,79,37,0,0,...,0,3.33,0.498,0,27.8,53.7,62.2,61.65,98,227
1,1.100000e+11,2,50,60,20,130,81,36,0,0,...,0,3.48,0.501,0,27.7,53.6,64.6,58.13,98,232
2,1.100000e+11,10,50,60,20,130,82,36,0,0,...,0,3.43,0.493,0,27.8,53.1,61.3,61.27,98,228
3,1.100000e+11,25,50,60,20,130,83,36,0,0,...,0,3.35,0.501,0,27.1,54.4,62.8,61.20,98,235
4,1.100000e+11,1,50,60,20,133,77,36,0,0,...,0,3.45,0.503,0,24.7,47.7,61.3,64.43,97,235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,1.100000e+11,10,50,65,15,130,79,38,0,0,...,0,3.39,0.543,0,25.5,4553.7,63.8,60.90,98,212
112,1.100000e+11,20,50,65,15,130,79,38,0,0,...,0,3.45,0.501,0,25.8,4553.7,64.9,62.33,98,204
113,1.100000e+11,2,50,60,15,130,82,41,0,0,...,0,3.28,0.507,0,29.0,4553.7,61.7,79.65,98,214
114,1.100000e+11,5,50,60,15,130,81,42,0,0,...,0,3.25,0.523,0,28.4,4553.7,62.9,70.88,98,195


In [5]:
#資料前處理
#去除掉數據為0，或是不相關的特徵
def clean(data):
    data = data.drop(["WorkNumber","SandWeight_E","SandWeight_L","SandWeight_9R","SandWeight_4C","SandWeight_9V","Resin_RA","Resin_PR877","Resin_K2032","Resin_PR853","Resin_RM893",
                      "Resin_RN815","Resin_KL1430L","Resin_Liquid","Hardener_40","Weight_RedIron"], axis=1)

    return data

In [6]:
#將X和Y資料取出。
clean_data = clean(data)
#Y的變數。
y_features = ["GrainSize\n(AFS)","AFS","UnityPoint","FallingTest"]
this_time_y_str = y_features[2]
this_time_y = clean_data[this_time_y_str]

In [7]:
# 用pearson corrleation做特徵選取(filter-based-method)，選取相關係數絕對值>=0.1以上的變數。
highly_corr_features = clean_data.columns[clean_data.corr()[this_time_y_str].abs()>=0.1]

clean_data2 = clean_data[highly_corr_features]
clean_data2

,MixingTime2,MixingTime3,TempSet,SandWeight_5V,SandWeight_5C,Resin_RB,Resin_KL130,Hardener_30,Water,Atmosphere_humidty,UnityPoint,FallingTest
0,60,20,130,54.0,210.5,4.62,2.34,5.01,3.33,53.7,98,227
1,60,20,130,52.5,210.0,4.65,2.34,5.05,3.48,53.6,98,232
2,60,20,130,53.0,207.5,4.63,2.27,4.98,3.43,53.1,98,228
3,60,20,130,52.5,206.0,4.62,2.30,4.99,3.35,54.4,98,235
4,60,20,133,0.0,0.0,7.01,2.35,5.07,3.45,47.7,97,235
...,...,...,...,...,...,...,...,...,...,...,...,...
111,65,15,130,0.0,203.0,6.82,2.32,5.16,3.39,4553.7,98,212
112,65,15,130,0.0,194.0,6.85,2.30,5.21,3.45,4553.7,98,204
113,60,15,130,0.0,238.0,6.74,2.30,5.09,3.28,4553.7,98,214
114,60,15,130,0.0,236.5,6.79,2.35,5.07,3.25,4553.7,98,195


In [8]:
# 利用train_test_split切分訓練集與測試集。

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(clean_data2,this_time_y,test_size=0.15, random_state=42)
X_train

,MixingTime2,MixingTime3,TempSet,SandWeight_5V,SandWeight_5C,Resin_RB,Resin_KL130,Hardener_30,Water,Atmosphere_humidty,UnityPoint,FallingTest
109,65,15,130,0.0,192.0,6.75,2.28,5.13,3.32,4553.7,99,200
100,65,15,130,0.0,194.5,6.72,2.30,5.13,3.32,4553.7,98,212
93,60,15,130,53.0,0.0,6.69,2.21,4.82,3.49,50.1,97,200
88,60,15,130,73.0,111.5,7.30,2.39,5.28,3.13,46.5,97,203
67,60,15,130,129.5,0.0,7.20,2.46,5.20,2.96,60.7,98,229
...,...,...,...,...,...,...,...,...,...,...,...,...
106,65,15,130,55.0,0.0,6.79,2.18,5.06,3.40,4553.7,98,208
14,60,20,138,0.0,0.0,6.64,2.26,4.99,3.32,41.7,98,239
92,60,15,130,0.0,0.0,7.26,2.40,5.28,3.08,51.6,96,218
51,60,15,128,100.0,112.5,7.02,2.35,5.08,3.34,58.8,98,217


In [9]:
#將訓練集的X做Standardization。
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
scaler = StandardScaler()
sc_x = scaler.fit_transform(X_train)
sc_x

array([[ 2.44948974, -0.39107694, -0.6033981 , ...,  1.85034233,
         1.48659418, -1.57357316],
       [ 2.44948974, -0.39107694, -0.6033981 , ...,  1.85034233,
         0.28257575, -0.62894731],
       [-0.40824829, -0.39107694, -0.6033981 , ..., -0.48565991,
        -0.92144268, -1.57357316],
       ...,
       [-0.40824829, -0.39107694, -0.6033981 , ..., -0.48488186,
        -2.1254611 , -0.15663439],
       [-0.40824829, -0.39107694, -1.17198478, ..., -0.48114725,
         0.28257575, -0.23535321],
       [ 2.44948974, -0.39107694, -0.6033981 , ...,  1.85034233,
         0.28257575, -0.9438226 ]])

In [10]:
# PCA沒有比較好，故不用。

# from sklearn.decomposition import PCA
# PCA_a = PCA(n_components=round(X_train.shape[1]*0.5))
# sc_x = PCA_a.fit_transform(sc_x)
# print(sc_x.shape)

In [11]:
#建模。

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import GridSearchCV

# model = XGBRegressor(n_estimators=100, random_state=42,verbose=1)
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model = SVR(kernel='linear')
# model = Ridge(random_state=42)

# 測試GridSearchCV效果。
xgb_param_grid = { 
    'n_estimators': [10, 100, 150, 200, 250, 300, 350, 400, 450, 500],
    'max_depth' : [3,4,5,6,7,8],
    'subsample':[0.5],
    'random_state':[42]
}

rf_param_grid = { 
    'n_estimators': [10, 100, 200, 300, 400, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [3,4,5,6,7,8],
    'criterion' :['squared_error', 'absolute_error', 'poisson'],
    'random_state':[42]
}

svr_param_grid = { 
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['auto', 'scale'],
    'C' : [0.1,1,10,100,20,50],
}

ridge_param_grid = { 
    'alpha': [1.0, 2.0,5.0,10.0],
    'solver' : ['auto','svd','cholesky','lsqr','sparse_cg','sag'],
    'random_state':[42],
}

rfc = RandomForestRegressor()
CV_rfc = GridSearchCV(estimator=rfc, param_grid=rf_param_grid, cv= 5)
CV_rfc.fit(sc_x, y_train)

model = RandomForestRegressor(**CV_rfc.best_params_)
model.fit(sc_x, y_train)

RandomForestRegressor(criterion='absolute_error', max_depth=3, n_estimators=10,
                      random_state=42)

In [12]:
#將測試集的X做Standardization。
X_test_sc = scaler.fit_transform(X_test)
# X_test_sc = PCA_a.fit_transform(X_test_sc)
X_test_sc

array([[-0.24253563, -0.53452248, -0.46544364,  0.96426653, -0.95180449,
         0.23785784,  0.31181324, -0.19281479,  0.16366994, -0.24872122,
        -0.90115511, -0.61821716],
       [-0.24253563,  1.87082869,  0.53991462, -1.15012016, -0.95180449,
         0.54096872,  1.01339303,  0.46826448,  1.00540107, -0.24707249,
        -0.90115511,  1.03302093],
       [-0.24253563, -0.53452248, -0.46544364,  0.96426653, -0.95180449,
        -0.00841975, -0.91595139, -1.43233841, -1.46718412, -0.23553138,
        -0.90115511,  0.24329837],
       [-0.24253563, -0.53452248, -0.46544364,  0.98598968, -0.95180449,
        -0.17891962, -1.26674129, -1.76287804, -1.41457592, -0.23524043,
        -2.37577257,  0.60226317],
       [-0.24253563,  1.87082869, -0.46544364,  0.27636675,  1.13273182,
         0.67357973,  1.89036777,  0.88143902, -1.25675134, -0.24018662,
         0.57346234,  0.53047021],
       [-0.24253563, -0.53452248, -0.46544364,  0.63117822, -0.95180449,
        -0.14103076, -

In [13]:
predicted_label = model.predict(X_test_sc)
predicted_label

array([97., 97., 97., 96., 98., 98., 98., 98., 98., 98., 98., 98., 97.,
       98., 99., 98., 97., 97.])

In [14]:
# 列出MSE、RMSE、MAE、R2。
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
MSE = mean_squared_error(predicted_label, y_test,squared=True)
RMSE = mean_squared_error(predicted_label, y_test,squared=False)
MAE = mean_absolute_error(predicted_label, y_test)
r2 = r2_score(predicted_label, y_test)
print("MSE= ",MSE)
print("RMSE= ",RMSE)
print("MAE= ",MAE)
print("R2= ",r2)

MSE=  0.0
RMSE=  0.0
MAE=  0.0
R2=  1.0
